### galton board


** Concept: ** (https://en.wikipedia.org/wiki/Bean_machine):

"The Galton Board consists of a vertical board with interleaved rows of pegs. Beads are dropped from the top and, when the device is level, bounce either left or right as they hit the pegs. Eventually they are collected into bins at the bottom, where the height of bead columns accumulated in the bins approximate a bell curve. Overlaying Pascal's triangle onto the pins shows the number of different paths that can be taken to get to each bin."

**Simulation:**
With the classes 'board', 'ball' and 'galton-board' a simulation of the Galton board can be started. 
The 'board' draws the containers and the corresponding obstacles in a triangular shape. Each sphere-object contains a random Bernoulli chain which determines the way through the obstacles. The way right past the obstacle is 1, the way left past the obstacle is 0.
The length of the chain is determined by the number of slots: lChain = nSlots -1
The number of obstacle rows corresponds to the length of the bernoulli chain.
 
**Parameter**

- nSlots = number of containers (standard = 10)
- nBalls = number of balls (standard = 100)
- nInterval = interval speed (standard = 1 ms)

In [1]:
# %matplotlib notebook
%matplotlib widget

import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.stats import bernoulli, norm


In [2]:
class board:
    def __init__(self, nSlots, ax, y0, xInterval, yInterval):
        self.ax = ax
        self.nSlots = nSlots
        self.res = np.zeros(nSlots)
        self.lenChain = nSlots - 1
        self.indexM = np.zeros([self.lenChain, (self.lenChain * 2 - 1)])
        #
        self.markerSize = 5
        self.xInterval = xInterval
        self.yInterval = yInterval
        self.xOffset = 5
        self.y_0 = y0
        #
        #
        self.xLim = [0, (self.xOffset + self.lenChain + self.xOffset -1)]
        self.yLim = [0, (self.y_0 * 1.20)]
        self.xm = 5
    #--------------------------------------------------------------------
    def initBoard(self):
        """ 
        Form matrix for triangle:
        the evenly distributed obstacles form a triangle.
        The matrix has the following size: chain length x (2* chain length -1), [2x3, 3x5, 4x7] etc.
        
                    ***+***  -> fill matrix line by line. 
                    **+*+**  -> start with last line and decrease 
                    *+*+*+*  -> index vector line by ine
                    +*+*+*+
        
         with the help of this matrix, the obstacles are drawn as circular markings in the figure
        """
        rowIndex = np.arange(0, self.indexM.shape[-1], 2)
        for i in list(range((self.lenChain - 1), -1, -1)):
            if i == (len(self.indexM)-1):
                self.indexM[i][rowIndex] = 1
                continue
            rowIndex = rowIndex[0:-1] +1
            self.indexM[i][rowIndex] = 1
        #
        # draw marks
        #
        x = self.xOffset
        y = self.y_0
        for i in range(0, len(self.indexM)):
            x = self.xOffset
            for c in range(0, self.indexM.shape[-1]):
                if self.indexM[i][c]:
                    self.ax.plot(x, y, 'ok', markersize=str(self.markerSize))
                x += self.xInterval
            y -= self.yInterval

        self.ax.set_xlim(self.xLim)
        self.ax.set_ylim(self.yLim)
        #
        # vertical lines to indicate the containers
#         ymax = 2
#         self.ax.vlines(x=(self.xOffset - self.xInterval), ymin=0, ymax= ymax, ls="-", lw=3)
#         self.ax.vlines(x=(self.xOffset + self.lenChain + self.xInterval - 1), ymin=0, ymax= ymax, ls="-", lw=3)
#         vx = np.arange(self.xOffset - 1, self.xOffset + self.lenChain, 1)
#         self.ax.vlines(x=vx, ymin=0, ymax =ymax, ls="-", lw=3)
        #
        # middle of figure in x-direction:
        self.xm = (self.ax.get_xlim()[-1] / 2)
        # No ticks
        self.ax.set_xticks([])
        self.ax.set_yticks([])


In [3]:
class ball:
    """
    A galton board contains x balls, which randomly fall through the obstacles and end up
    in a container. The path of the balls is given by a random Bernoulli series. The balls run
    one after the other through the board.

    Members:
    - x: x-coordinate of ball
    - y: y-coordinate of ball
    - xInterval: value of distance between obstacles in x direction 
    - yInterval: value of distance between obstacles in y direction 
    - color: line color
    - line: lin2D objekt das die Kugel im Diagram representiert
    - gltWay: Bernoulli-series with lenght: nSlots-1
    - gltLen: size of Bernoullie-series
        
    Methods:
    - move_right(): move ball right
    - move_left(): move ball left
    """
    def __init__(self, ax, gltWay, y0, xInterval, yInterval):
        self.x = (ax.get_xlim()[-1] / 2)
        self.y = y0 + yInterval
        self.xInterval = xInterval
        self.yInterval = yInterval
        self.line, = ax.plot(self.x, self.y, 'or', markersize="8")
        self.color = 'red'
        self.gltWay = gltWay
        self.gltLen = len(gltWay)
        self.runIndex = 0
    #
    def move_right(self):
        if isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0][0]
            self.y = self.line.get_data()[-1][0]
            self.line.set_data(self.x + self.xInterval, self.y - 2)
            return
        if not isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0]
            self.y = self.line.get_data()[-1]
            self.line.set_data(self.x + self.xInterval, self.y - self.yInterval)
        return
    #
    def move_left(self):
        if isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0][0]
            self.y = self.line.get_data()[-1][0]
            self.line.set_data(self.x - self.xInterval, self.y - 2)
            return
        if not isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0]
            self.y = self.line.get_data()[-1]
            self.line.set_data(self.x - self.xInterval, self.y - self.yInterval)
        return

In [4]:
#-----------------------------------------------------------------------------------
class GaltonBoard:
    """ 
    class GaltonBoard: Parent class to make the control of the animation more clearly arranged.
                        Class contains as member the board and the balls.
    
    Members:
    - board:      board-objekt 
    - balls:      ball-objekt
    - activeBall: index value of the current ball runningn through the board
    - ax:         ax-objekt of diagram
    - ax2:        2nd ax-objekt für 2nd y-axis. used for fitting. 
    
    Methods:
    - getBall(iBall):  return specific ball
    - getBallsCount(): return total number of balls
    - getSlotsCount(): return number of containers
    - getAxis():       return ax-objeckt of figure
    - clearBall(iBall):    let ball disappear from figure
    - AddBallResult(iResult): increment the container wher the ball falls in
    - DrawFittingFunction(x,y): draw a normal distribution fitted to the containers content
    
    """ 
    def __init__(self, ax, nSlots, nBalls):
        self.balls = []
        self.activeBall = 0
        self.ax = ax
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        self.text = ax.text(0.05, 0.15,'0', transform=self.ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
        self.ax2 = self.ax.twinx()
        self.ax2.set_yticks([])
        self.xInterval = 0.5
        self.yInterval = 1.5
        #
        if nSlots > 10:
            y0 = 15 * self.yInterval
        else:
            y0 = 15       
        #
        if nSlots > 10 or nBalls > 20:
            y0 = 15 + 0.30*nBalls
        #
        self.board = board(nSlots, ax, y0, self.xInterval, self.yInterval)
        #
        # initialize figure:
        self.board.initBoard()
        #
        # save balls in a list
        lenWay = nSlots-1
        self.result = []
        for i in range(nBalls):
            self.balls.append(ball(ax, bernoulli.rvs(0.5, size = lenWay), self.board.y_0, self.board.xInterval, self.board.yInterval))
            self.result.append(np.sum(self.balls[i].gltWay))
    #
    def getBall(self, index):
        return self.balls[index]
    #
    def getBallsCount(self):
        return len(self.balls)
    #
    def getSlotsCount(self):
        return self.board.nSlots
    #
    def getAxis(self):
        return self.board.ax
    #
    def clearBall(self, iBall):
        self.balls[iBall].line.remove()
    #
    def AddBallResult(self, iResult):
        self.board.res[iResult] += 1
        iMax = self.board.lenChain + self.board.xOffset
        x = np.arange(self.board.xOffset -1, iMax, 1)
        self.rects = self.board.ax.bar(x, self.board.res , color="blue")
#         self.board.ax.bar(self.board.xOffset - self.board.xInterval, self.board.res[0], color="blue" )
#         self.board.ax.bar(x, self.board.res[1:len(x)+1] , color="blue")
#         self.board.ax.bar(iMax - self.board.xInterval, self.board.res[-1],color="blue")
        #
    #
    def DrawFittingFunction(self, x, y):
        # draw fitting with a second y-axis
        #
        # höhe der einzelnen behälter anzeigen:
        #
        for rect in self.rects:
            height = rect.get_height()
            self.board.ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,'%d' % int(height),ha='center', va='bottom')
        #
        self.ax2.plot(x, y, 'C1')        
        self.ax2.set_ylim(0, np.amax(y) * 1.5)
#-----------------------------------------------------------------------------------

In [5]:
def run_galton(*args):
    """
  update function of the animation
    - leads each ball through the obstacles and stores the result as a bar-plot
    - ends the animation when all balls have passed through the obstacles and
      fits a normal distribution from the result
    """
    #
    # end of animation when all balls have run through the obstacles
    if galton.activeBall >= galton.getBallsCount(): 
        #
        #fit result (container contents) with a normal distribution
        #
        # - the resulting distribution is set to the middle of the diagram (x-direction).
        #
        fitting = norm.fit(galton.result)
        
        xFit =  np.arange(galton.ax.get_xlim()[0], galton.ax.get_xlim()[-1], 1/200)
        # wenn mu eine ganze Zahl ist, wird die Normalverteilung in der Mitte angezeigt, andernfalls +/- xIntervall versetzt.
        #
#         if (galton.board.nSlots/2 - fitting[0]) == 0:
#             pdfFit = norm.pdf(xFit, galton.board.xm, fitting[1])
#         #
#         if (galton.board.nSlots/2 - fitting[0]) > 0:
#             pdfFit = norm.pdf(xFit, galton.board.xm - galton.board.xInterval, fitting[1])
#         #
#         if (galton.board.nSlots/2 - fitting[0]) < 0:
#             pdfFit = norm.pdf(xFit, galton.board.xm + galton.board.xInterval, fitting[1])
        #
        pdfFit = norm.pdf(xFit, galton.board.xm, fitting[1])
        #
        textstr = '\n'.join((\
                             "parameter of gauss distribution",
                             r'$\mu=%.2f$' % (fitting[0], ),
                             r'$\mathrm{\sigma}=%.2f$' % (fitting[1], ),
                             r'$\mathrm{max =}%.2f$' %(np.amax(pdfFit), )
                             ))
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        galton.ax.text(0.05, 0.95, textstr, transform=galton.ax.transAxes, fontsize=10,\
                       verticalalignment='top', bbox=props)
        #
        galton.DrawFittingFunction(xFit, pdfFit)
        #
        #
        anim.event_source.stop()
        return galton
        #
    #
    # get balls object which should run 
    ActBall = galton.getBall(galton.activeBall)
    #
    # is current ball already through the obstacles?
    if ActBall.runIndex >= (ActBall.gltLen - 1):
    #
        # increase containers content. 
        galton.AddBallResult(np.sum(ActBall.gltWay))
        # delete ball from diagram
        galton.clearBall(galton.activeBall)
        # increase balls counter index
        galton.activeBall += 1
        return ActBall.line
    #
    # move balls left/right around the obstacles
    if ActBall.runIndex < ActBall.gltLen:
        if ActBall.gltWay[ActBall.runIndex] > 0:
            ActBall.move_right()
        #
        if ActBall.gltWay[ActBall.runIndex] == 0:
            ActBall.move_left()
        #
    ActBall.runIndex += 1
    galton.text.set_text(r'$\Sigma = %d$' %(galton.activeBall +1, ))

    
    return ActBall.line


In [6]:
# parameter of simulation
#
nSlots = 13
nBalls = 200
nInterval = 1
#

In [7]:
# start animation with given parameters
# 
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
#
galton = GaltonBoard(axes, nSlots, nBalls)
#
anim = FuncAnimation(fig, run_galton, interval=nInterval, blit=True)
#
# historgram and gaussian-fit
#
# axes[1].hist(galton.result, bins=nSlots, density=True, label='experiment')
#
fitting = norm.fit(galton.result)
# xLim = axes[1].get_xlim()
# x2 = np.arange(xLim[0], xLim[-1], 1/200)
# axes[1].plot(x2, norm.pdf(x2, fitting[0], fitting[1]), label='gaussian fit')
#
# axes[1].set_xticks([])
# axes[1].set_yticks([])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Copyright © 2020 IUBH Internationale Hochschule